In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import hanja
import pinyin
import py

In [2]:
def char_info(char):
    info = {'char': char}
    response = requests.get(f'https://ytenx.org/zim?dzih={char}&dzyen=1&jtkb=1&jtkd=1&jtdt=1&jtgt=1')
    try: 
        soup = BeautifulSoup(response.text, 'lxml')
        entry = list(soup.find_all('p')[0])
        if entry[0] == '\n':
            return pd.DataFrame(info, index=[0])
        info['tone'] = entry[0][-1]
        info['rime'] = entry[1].text
        if info['tone'] == '入':
            info['ipa'] = entry[4].text.split()[1][0:-1]
            info['tone_class'] = 'E'
        elif info['tone'] == '平':
            info['ipa'] = entry[4].text.split()[1][0:-1]
            info['tone_class'] = 'L'        
        else:
            info['ipa'] = entry[4].text.split()[1][0:-2]
            info['tone_class'] = entry[4].text.split()[1][-2]
        return pd.DataFrame(info, index=[0])
    except:
        return pd.DataFrame()

In [3]:
char_info('明')

,char,tone,rime,ipa,tone_class
0,明,平,庚,mjaeng,L


In [4]:
char_info('起')

,char,tone,rime,ipa,tone_class
0,起,上,止,khi,X


In [5]:
char_info('會')

,char,tone,rime,ipa,tone_class
0,會,去,泰,hwaj,H


In [6]:
char_info('烈')

,char,tone,rime,ipa,tone_class
0,烈,入,薛,ljet,E


In [9]:
df = pd.read_csv('../../data/char_list.csv', index_col=0)

In [10]:
df

,char
0,㑹
1,㒿
2,㕙
3,㕮
4,㖀
...,...
8563,𫇢
8564,𬋖
8565,𮪃
8566,𮫃


In [11]:
%%time
dictionary = pd.concat([char_info(i) for i in df.char])

Wall time: 30min 9s


In [13]:
dictionary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8443 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   char        8443 non-null   object
 1   tone        8010 non-null   object
 2   rime        8010 non-null   object
 3   ipa         8010 non-null   object
 4   tone_class  8010 non-null   object
dtypes: object(5)
memory usage: 395.8+ KB


In [14]:
dictionary.to_csv(f'../../data/dictionary.csv')